In [1]:
# imports
import numpy as np
import os

from numpy.random import default_rng
# stabilité du notebook d'une exécution à l'autre
random=default_rng(42) 

# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

## Import csv

In [2]:
import pandas as pd

#Récupération des données
general_data = pd.read_csv('Donnees/general_data.csv')
employee_survey_data = pd.read_csv('Donnees/employee_survey_data.csv')
manager_survey_data = pd.read_csv('Donnees/manager_survey_data.csv')
in_time = pd.read_csv('Donnees/in_time.csv')
out_time = pd.read_csv('Donnees/out_time.csv')

## Combinaison DataFrame

In [3]:
#fusion des données
general_employee = pd.merge(employee_survey_data, general_data, on='EmployeeID', how='inner')
general_employee_manager = pd.merge(general_employee, manager_survey_data, on='EmployeeID', how='inner')

general_employee_manager=general_employee_manager.dropna().reset_index(drop=True)

#suppression des colonnes inutiles
general_employee_manager = general_employee_manager.drop('Over18', axis=1)
general_employee_manager = general_employee_manager.drop('EmployeeID', axis=1)

#affichage des données
general_employee_manager

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,...,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating
0,3.0,4.0,2.0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,...,11,8,0,1.0,6,1,0,0,3,3
1,3.0,2.0,4.0,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,...,23,8,1,6.0,3,5,1,4,2,4
2,2.0,2.0,1.0,32,No,Travel_Frequently,Research & Development,17,4,Other,...,15,8,3,5.0,2,5,0,3,3,3
3,4.0,4.0,3.0,38,No,Non-Travel,Research & Development,2,5,Life Sciences,...,11,8,3,13.0,5,8,7,5,2,3
4,4.0,1.0,3.0,32,No,Travel_Rarely,Research & Development,10,1,Medical,...,12,8,2,9.0,2,6,0,4,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4295,3.0,4.0,3.0,29,No,Travel_Rarely,Sales,4,3,Other,...,18,8,0,6.0,2,6,1,5,2,3
4296,4.0,1.0,3.0,42,No,Travel_Rarely,Research & Development,5,4,Medical,...,17,8,1,10.0,5,3,0,2,3,3
4297,4.0,4.0,3.0,29,No,Travel_Rarely,Research & Development,2,4,Medical,...,15,8,0,10.0,2,3,0,2,2,3
4298,1.0,3.0,3.0,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,...,20,8,0,5.0,4,4,1,2,3,4


## Version Thib

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in split.split(general_employee_manager, general_employee_manager["Attrition"]):
    stratified_train_set = general_employee_manager.loc[train_index]
    stratified_test_set = general_employee_manager.loc[test_index]

general_employee_manager_labels = stratified_train_set["Attrition"].copy()

for set_ in (stratified_train_set, stratified_test_set):
    set_.drop("Attrition", axis=1, inplace=True)

general_employee_manager_prepared = stratified_train_set.copy()

## Test

In [145]:
from sklearn.model_selection import train_test_split

#suppression des colonnes inutiles
general_employee_manager = general_employee_manager.drop('BusinessTravel', axis=1)
general_employee_manager = general_employee_manager.drop('Department', axis=1)
general_employee_manager = general_employee_manager.drop('EducationField', axis=1)
general_employee_manager = general_employee_manager.drop('Gender', axis=1)
general_employee_manager = general_employee_manager.drop('JobRole', axis=1)
general_employee_manager = general_employee_manager.drop('MaritalStatus', axis=1)

#création des données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(general_employee_manager.drop('Attrition',axis=1), 
                                                    general_employee_manager['Attrition'], test_size=0.30, 
                                                    random_state=101)

In [146]:
from sklearn.linear_model import LogisticRegression

#Utilisation de la régression logistique
lr=LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

#affichage de la précision
print("score on test: " + str(lr.score(X_test, y_test)))
print("score on train: "+ str(lr.score(X_train, y_train)))

score on test: 0.8302325581395349
score on train: 0.8418604651162791


In [147]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier 
# max_samples: maximum size 0.5=50% of each sample taken from the full dataset
# max_features: maximum of features 1=100% taken here all 10K 
# n_estimators: number of decision trees

#Utilisation de l'arbre de décision
bg=BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10)
bg.fit(X_train, y_train)

#affichage de la précision
print("score on test: " + str(bg.score(X_test, y_test)))
print("score on train: "+ str(bg.score(X_train, y_train)))

score on test: 0.92015503875969
score on train: 0.9754152823920266


In [148]:
from sklearn.ensemble import RandomForestClassifier
# n_estimators = number of decision trees

#uttilisation de la forêt aléatoire
rf = RandomForestClassifier(n_estimators=30, max_depth=9)
rf.fit(X_train, y_train)

#affichage de la précision
print("score on test: " + str(rf.score(X_test, y_test)))
print("score on train: "+ str(rf.score(X_train, y_train)))

score on test: 0.8953488372093024
score on train: 0.9332225913621263


## Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

general_employer_manager_num = general_employee_manager_prepared.select_dtypes(include=[np.number])
num_attribs = list(general_employer_manager_num)
cat_attribs = ["Attrition", "BusinessTravel", "Department", "EducationField","Gender","JobRole","MaritalStatus", ]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

general_employee_manager_prepared = full_pipeline.fit_transform(general_employee_manager_prepared)

general_employee_manager_prepared